# Speech Understanding
## Assignment 2
#### Question 1.2
### Akansha Gautam (M23CSA506)

# Import libraries

In [163]:
import numpy as np
import pandas as pd
import torch
import subprocess
from transformers import Wav2Vec2Processor, HubertForCTC
from sklearn.metrics import roc_curve
from transformers import AutoModel, AutoProcessor, AutoFeatureExtractor
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
import torch.nn.functional as F
import IPython.display as ipd
import librosa.display
import torchaudio
import torchaudio.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, WavLMModel
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.mixture import GaussianMixture
from sklearn.metrics import confusion_matrix
from transformers import UniSpeechSatModel, UniSpeechSatConfig, Wav2Vec2FeatureExtractor
from scipy.interpolate import interp1d
from scipy.optimize import brentq
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from pytube import YouTube
import ffmpeg
import os
import loralib as lora
from loralib import Linear as LoRALinear

pd.set_option('display.max_colwidth', None)

# Set Device

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {device}")

Using device: cpu


# Load vox1 audio file

In [19]:
vox1_path = "/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1"

data = []

for root, dirs, files in os.walk(vox1_path):
    for file in files:
        if file.endswith(".wav"):
            # path = os.path.join(dirs)
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, os.path.join(vox1_path, "wav"))
            data.append({"file_name": file, "abs_filepath": full_path, "rel_filepath": rel_path})

df_vox1 = pd.DataFrame(data)
print(f"vox1 shape: {df_vox1.shape}")
df_vox1.head()

vox1 shape: (4874, 3)


,file_name,abs_filepath,rel_filepath
0,00005.wav,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1/wav/id10295/nt7dNRvlEHE/00005.wav,id10295/nt7dNRvlEHE/00005.wav
1,00004.wav,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1/wav/id10295/nt7dNRvlEHE/00004.wav,id10295/nt7dNRvlEHE/00004.wav
2,00001.wav,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1/wav/id10295/nt7dNRvlEHE/00001.wav,id10295/nt7dNRvlEHE/00001.wav
3,00003.wav,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1/wav/id10295/nt7dNRvlEHE/00003.wav,id10295/nt7dNRvlEHE/00003.wav
4,00002.wav,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1/wav/id10295/nt7dNRvlEHE/00002.wav,id10295/nt7dNRvlEHE/00002.wav


# Load the list of trial pairs of vox celeb1

In [20]:
# 1 if the two audio clips are from the same speaker, 0 otherwise

vox_celeb1_trial_pairs = '/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/list_of_trial_pairs_vox_celeb1.txt'

df_vox_celeb1_trial_pairs = pd.read_csv(vox_celeb1_trial_pairs, sep=' ', header=None)
df_vox_celeb1_trial_pairs.columns = ['label', 'path1', 'path2']

print(f"vox1 shape: {df_vox_celeb1_trial_pairs.shape}")
df_vox_celeb1_trial_pairs.head()

vox1 shape: (37611, 3)


,label,path1,path2
0,1,id10270/x6uYqmx31kE/00001.wav,id10270/8jEAjG6SegY/00008.wav
1,0,id10270/x6uYqmx31kE/00001.wav,id10300/ize_eiCFEg0/00003.wav
2,1,id10270/x6uYqmx31kE/00001.wav,id10270/GWXujl-xAVM/00017.wav
3,0,id10270/x6uYqmx31kE/00001.wav,id10273/0OCW1HUxZyg/00001.wav
4,1,id10270/x6uYqmx31kE/00001.wav,id10270/8jEAjG6SegY/00022.wav


# Load audio files with trial pairs of vox celeb1

In [21]:
df_pairs_with_path1 = df_vox_celeb1_trial_pairs.merge(
    df_vox1[['rel_filepath', 'file_name', 'abs_filepath']],
    left_on='path1',
    right_on='rel_filepath',
    how='left'
).rename(columns={'file_name': 'path1_file_name', 'abs_filepath': 'path1_abs_filepath'}).drop(columns=['rel_filepath'])

df_final_pairs = df_pairs_with_path1.merge(
    df_vox1[['rel_filepath', 'file_name', 'abs_filepath']],
    left_on='path2',
    right_on='rel_filepath',
    how='left'
).rename(columns={'file_name': 'path2_file_name', 'abs_filepath': 'path2_abs_filepath'}).drop(columns=['rel_filepath'])

df_final_pairs = df_final_pairs[
    ['label', 'path1', 'path1_file_name', 'path1_abs_filepath', 'path2', 'path2_file_name', 'path2_abs_filepath']
]

print(f"Final merged shape: {df_final_pairs.shape}")
df_final_pairs.head(2)


Final merged shape: (37611, 7)


,label,path1,path1_file_name,path1_abs_filepath,path2,path2_file_name,path2_abs_filepath
0,1,id10270/x6uYqmx31kE/00001.wav,00001.wav,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1/wav/id10270/x6uYqmx31kE/00001.wav,id10270/8jEAjG6SegY/00008.wav,00008.wav,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1/wav/id10270/8jEAjG6SegY/00008.wav
1,0,id10270/x6uYqmx31kE/00001.wav,00001.wav,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1/wav/id10270/x6uYqmx31kE/00001.wav,id10300/ize_eiCFEg0/00003.wav,00003.wav,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/Speech2025Datasets/vox1/wav/id10300/ize_eiCFEg0/00003.wav


# Load Pre-trained processor and model

In [133]:
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
model.eval()
model = model.to(device)

# Perform Speaker Verification (Evaluation) using the list of trial pairs - VoxCeleb1 (cleaned) file

In [39]:
# Reduced the size of trial pairs to 500
df_final = df_final_pairs.sample(frac=1, random_state=42).reset_index(drop=True)
df_final = df_final.head(500)
df_final.shape

(500, 7)

In [134]:
def extract_embedding(audio_path):

    waveform, _ = librosa.load(audio_path, sr=16000)

    if waveform.ndim > 1:
        waveform = waveform[0]

    inputs = feature_extractor(waveform, sampling_rate=16000, return_tensors="pt")
    input_values = inputs["input_values"].to(device)

    with torch.no_grad():
        outputs = model(input_values)
        hidden_states = outputs.last_hidden_state

    embedding = hidden_states.mean(dim=1).squeeze().cpu().numpy()
    
    return embedding

y_true = []
y_pred = []
similarities = []

for _, row in tqdm(df_final.iterrows(), total=len(df_final)):
    emb1 = extract_embedding(row['path1_abs_filepath'])
    emb2 = extract_embedding(row['path2_abs_filepath'])

    similarity = cosine_similarity([emb1], [emb2])[0][0]
    similarities.append(similarity)
    predicted_label = 1 if similarity > 0.7 else 0

    y_true.append(row['label'])
    y_pred.append(predicted_label)


100%|██████████| 500/500 [08:41<00:00,  1.04s/it]


In [135]:
# Calculate Speaker Verification Accuracy
accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print(f"Speaker verification accuracy using UniSpeech-SAT model: {accuracy}")

# Calculate Equal Error Rate (ERR) in %
false_positive_rate, true_positive_rate, _ = roc_curve(y_true, similarities)
false_negative_rate = 1 - true_positive_rate
eer_function = interp1d(false_positive_rate, false_negative_rate)
eer = (brentq(lambda x: eer_function(x) - x, 0.0, 1.0) * 100)
print(f"Equal Error Rate in % using UniSpeech-SAT model: {accuracy}")

# Calcualte TAR@1% FAR
target_far = 0.01
tar_at_1_far = 0.0
for i in range(len(false_positive_rate)):
    if false_positive_rate[i] <= target_far:
        tar_at_1_far = true_positive_rate[i] 

tar_at_1_far *= 100 
print(f"TAR@1% FAR using UniSpeech-SAT model: {accuracy}")

Speaker verification accuracy using UniSpeech-SAT model: 0.488
Equal Error Rate in % using UniSpeech-SAT model: 0.488
TAR@1% FAR using UniSpeech-SAT model: 0.488


# Fine-tune the speaker verification model using LoRA and ArcFace loss with voxceleb2 dataset

In [136]:
# Load the vox celeb2 audio files path
vox2_path = '/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac'
data = []

for root, dirs, files in os.walk(vox2_path):
    for file in files:
        if file.endswith(".m4a"):
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, os.path.join(vox2_path, "m4a"))
            parts = full_path.split(os.sep)
            identity = parts[parts.index('aac') + 1]
            data.append({"file_name": file, "abs_filepath": full_path, "rel_filepath": rel_path, "identity": identity})

df_vox2 = pd.DataFrame(data)
df_vox2 = df_vox2.sort_values(by="identity", ascending=True, ignore_index=True)
print(f"vox2 shape: {df_vox2.shape}")
df_vox2.head(2)

vox2 shape: (36237, 4)


,file_name,abs_filepath,rel_filepath,identity
0,00043.m4a,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id00017/OLguY5ofUrY/00043.m4a,../id00017/OLguY5ofUrY/00043.m4a,id00017
1,00179.m4a,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id00017/lvX8diWgriA/00179.m4a,../id00017/lvX8diWgriA/00179.m4a,id00017


In [137]:
# Load the waveform of voxCeleb2 audio files
waveforms = []
sampling_rates = []
for index, row in df_vox2.iterrows():
    abs_filepath = row['abs_filepath']
    try:
        waveform, sr = librosa.load(abs_filepath, sr=16000)
        waveforms.append(waveform)
        sampling_rates.append(sr)
    except Exception as e:
        print(f"Failed to load file: {abs_filepath}")
        waveforms.append([])
        sampling_rates.append(0)

df_vox2['waveform'] = waveforms
df_vox2['sampling_rate'] = sampling_rates

/var/folders/12/7zfx6db53p5bf6btn6s9bqy40000gn/T/ipykernel_9574/2654778886.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  waveform, sr = librosa.load(abs_filepath, sr=16000)
/Users/akanshagautam/Documents/MTech/ml_env/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id02317/d4dinqezkKo/00383.m4a
Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id03030/haoNit7a4W0/00202.m4a
Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id03862/CJ8_3dVx7M4/00134.m4a
Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id04094/DRq5F2261Ko/00076.m4a
Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id04570/uLdhmrd9ULA/00369.m4a


In [170]:
# Split voxceleb2 dataset into train and test
all_speakers = sorted(df_vox2['identity'].unique())

train_speakers = all_speakers[:100]
test_speakers = all_speakers[100:]
print("total train speakers:", len(train_speakers))
print("total test speakers:", len(test_speakers))

spk2label = {spk: i for i, spk in enumerate(train_speakers)}

df_train_vox2 = df_vox2[df_vox2['identity'].isin(train_speakers)].reset_index(drop=True)
df_test_vox2 = df_vox2[df_vox2['identity'].isin(test_speakers)].reset_index(drop=True)
print("train dataframe shape:", df_train_vox2.shape)
print("test datafram shape:", df_test_vox2.shape)


total train speakers: 100
total test speakers: 18
train dataframe shape: (29831, 6)
test datafram shape: (6406, 6)


In [171]:
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
model = model.to(device)

for layer in model.encoder.layers:
    layer.attention.q_proj = LoRALinear(layer.attention.q_proj.in_features, layer.attention.q_proj.out_features)
    layer.attention.v_proj = LoRALinear(layer.attention.v_proj.in_features, layer.attention.v_proj.out_features)

for name, param in model.named_parameters():
    if 'lora' in name:
        param.requires_grad = True

In [172]:
def init_arcface_weights(embedding_dim, num_classes):
    W = torch.nn.Parameter(torch.randn(embedding_dim, num_classes))
    torch.nn.init.xavier_uniform_(W)
    return W

def arcface_loss_fn(embeddings, labels, W, scale=30.0, margin=0.5):
    embeddings = F.normalize(embeddings)
    W = F.normalize(W, dim=0)
    cosine = torch.matmul(embeddings, W)
    theta = torch.acos(torch.clamp(cosine, -1.0 + 1e-7, 1.0 - 1e-7))
    target_logits = torch.cos(theta + margin)
    one_hot = F.one_hot(labels, num_classes=W.shape[1]).float().to(embeddings.device)
    logits = cosine * (1 - one_hot) + target_logits * one_hot
    logits *= scale
    return F.cross_entropy(logits, labels)

arcface_W = init_arcface_weights(model.config.hidden_size, len(train_speakers)).to(device)
optimizer = torch.optim.AdamW(list(model.parameters()) + [arcface_W], lr=1e-4)

In [173]:
num_epochs = 5
max_duration_sec = 3
sample_rate = 16000
max_len = sample_rate * max_duration_sec
batch_size = 8

df_train_vox2_updated = df_train_vox2.head(1000)

for epoch in range(num_epochs):
    print(f"---- Epoch {epoch+1} ----")
    total_loss = 0.0
    model.train()

    for i in tqdm(range(0, len(df_train_vox2_updated), batch_size)):
        batch_rows = df_train_vox2_updated.iloc[i:i+batch_size]
        input_tensors = []
        label_tensors = []

        for _, row in batch_rows.iterrows():
            waveform = row['waveform']
            identity = row['identity']

            if len(waveform) == 0 or identity not in spk2label:
                continue

            if len(waveform) > max_len:
                waveform = waveform[:max_len]
            else:
                waveform = np.pad(waveform, (0, max_len - len(waveform)))

            inputs = feature_extractor(torch.tensor(waveform), sampling_rate=sample_rate, return_tensors="pt")
            input_tensors.append(inputs["input_values"].squeeze(0))
            label_tensors.append(spk2label[identity])

        if not input_tensors:
            continue

        inputs_batch = torch.stack(input_tensors).to(device)
        labels_batch = torch.tensor(label_tensors).to(device)

        outputs = model(input_values=inputs_batch)
        embeddings = outputs.last_hidden_state.mean(dim=1)

        loss = arcface_loss_fn(embeddings, labels_batch, arcface_W)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / (len(df_train_vox2_updated) // batch_size)
    print(f"Avg Loss: {avg_loss:.4f}")

---- Epoch 1 ----


100%|██████████| 125/125 [11:13<00:00,  5.39s/it]


Avg Loss: 8.1646
---- Epoch 2 ----


100%|██████████| 125/125 [11:06<00:00,  5.33s/it]


Avg Loss: 8.0290
---- Epoch 3 ----


...

Avg Loss: 15.6219
---- Epoch 4 ----


100%|██████████| 125/125 [10:49<00:00,  5.20s/it]


Avg Loss: 16.9534
---- Epoch 5 ----


100%|██████████| 125/125 [10:53<00:00,  5.23s/it]

Avg Loss: 14.0448


In [174]:
# Test the fine-tuned model on vox-celeb-1 dataset

def extract_embedding(audio_path):

    waveform, _ = librosa.load(audio_path, sr=16000)

    if waveform.ndim > 1:
        waveform = waveform[0]

    inputs = feature_extractor(waveform, sampling_rate=16000, return_tensors="pt")
    input_values = inputs["input_values"].to(device)

    with torch.no_grad():
        outputs = model(input_values)
        hidden_states = outputs.last_hidden_state

    embedding = hidden_states.mean(dim=1).squeeze().cpu().numpy()
    
    return embedding

y_true = []
y_pred = []
similarities = []

for _, row in tqdm(df_final.iterrows(), total=len(df_final)):
    emb1 = extract_embedding(row['path1_abs_filepath'])
    emb2 = extract_embedding(row['path2_abs_filepath'])

    similarity = cosine_similarity([emb1], [emb2])[0][0]
    similarities.append(similarity)
    predicted_label = 1 if similarity > 0.7 else 0

    y_true.append(row['label'])
    y_pred.append(predicted_label)


100%|██████████| 500/500 [15:19<00:00,  1.84s/it]


In [175]:
# Calculate Speaker Verification Accuracy
accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print(f"Speaker verification accuracy using UniSpeech-SAT model: {accuracy}")

# Calculate Equal Error Rate (ERR) in %
false_positive_rate, true_positive_rate, _ = roc_curve(y_true, similarities)
false_negative_rate = 1 - true_positive_rate
eer_function = interp1d(false_positive_rate, false_negative_rate)
eer = (brentq(lambda x: eer_function(x) - x, 0.0, 1.0) * 100)
print(f"Equal Error Rate in % using UniSpeech-SAT model: {accuracy}")

# Calcualte TAR@1% FAR
target_far = 0.01
tar_at_1_far = 0.0
for i in range(len(false_positive_rate)):
    if false_positive_rate[i] <= target_far:
        tar_at_1_far = true_positive_rate[i] 

tar_at_1_far *= 100 
print(f"TAR@1% FAR using UniSpeech-SAT model: {accuracy}")

Speaker verification accuracy using UniSpeech-SAT model: 0.5
Equal Error Rate in % using UniSpeech-SAT model: 0.5
TAR@1% FAR using UniSpeech-SAT model: 0.5


: 